<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/Upside_Down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/BY571/Upside-Down-Reinforcement-Learning
import numpy as np
import torch, copy
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import torch.nn.functional as F
import matplotlib.pyplot as plt
!pip install gym[box2d]
import gym

# https://github.com/openai/gym/issues/1898#issuecomment-860034155
!apt-get install python-opengl -y xvfb
!pip install pyvirtualdisplay 
from pyvirtualdisplay import Display
# https://github.com/lucidrains/perceiver-pytorch
!pip install perceiver-pytorch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# init Environment
# env = gym.make("CartPole-v0")
env = gym.make("LunarLander-v2")
action_space = env.action_space.n
state_space = env.observation_space.shape[0]
max_reward = 200 #CartPole-v0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

state = env.reset()
print("state",state,state.shape)
action = env.action_space.sample() # Take a random action
observation, reward, done, info = env.step(action)
print(observation)
display = Display(visible=0, size=(1400, 900))
display.start()
screen = env.render('rgb_array')
# plt.imshow(screen)
# plt.grid(False)
# print(screen.shape) #(400, 600, 3)

horizon_scale = 0.02
return_scale = 0.02
replay_size = 700
n_warm_up_episodes = 50
n_updates_per_iter = 100
n_episodes_per_iter = 15
last_few = 50
batch_size = 256

state [ 0.00205469  1.4062573   0.20809956 -0.20723048 -0.00237407 -0.04713768
  0.          0.        ] (8,)
[ 0.00410919  1.4010174   0.20781843 -0.23288482 -0.00470341 -0.04659079
  0.          0.        ]


In [3]:
# # Behavior Function
# class BF(nn.Module):
#     def __init__(self, state_space, action_space, hidden_size, seed):
#         super(BF, self).__init__()
#         torch.manual_seed(seed)
#         self.actions = np.arange(action_space)
#         self.action_space = action_space
#         self.fc1 = nn.Linear(state_space, hidden_size)
#         self.commands = nn.Linear(2, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.fc3 = nn.Linear(hidden_size, hidden_size)
#         self.fc4 = nn.Linear(hidden_size, hidden_size)
#         self.fc5 = nn.Linear(hidden_size, action_space)
#         self.sigmoid = nn.Sigmoid()
         
#     def forward(self, state, command):
#         out = self.sigmoid(self.fc1(state))
#         command_out = self.sigmoid(self.commands(command))
#         out = out * command_out
#         out = torch.relu(self.fc2(out))
#         out = torch.relu(self.fc3(out))
#         out = torch.relu(self.fc4(out))
#         out = self.fc5(out)
#         return out
    
#     def action(self, state, desire, horizon):
#         """Samples the action based on their probability"""
#         command = torch.cat((desire*return_scale,horizon*horizon_scale), dim=-1)
#         action_prob = self.forward(state, command)
#         probs = torch.softmax(action_prob, dim=-1)
#         m = Categorical(probs)
#         action = m.sample()
#         return action

#     def greedy_action(self, state, desire, horizon):
#         command = torch.cat((desire*return_scale,horizon*horizon_scale), dim=-1)
#         action_prob = self.forward(state, command)
#         probs = torch.softmax(action_prob, dim=-1)
#         action = torch.argmax(probs).item()
#         return action



from perceiver_pytorch.perceiver_io import *

# class PerceiverIO(nn.Module):
#     def __init__():
class BF(nn.Module):
    # def __init__(self, state_space, action_space, hidden_size, seed,
    def __init__(self, dim, latent_dim, hidden_size=0, seed=0,
        # self,
        *,
        depth=6,
        # dim,
        queries_dim=32,
        logits_dim = None,
        num_latents = 1,#512,
        # latent_dim = 7,
        cross_heads = 1,
        latent_heads = 8,
        cross_dim_head = 64,
        latent_dim_head = 64,
        weight_tie_layers = False,
        decoder_ff = False
    ):
        dim+=2
        print("dim",dim)
        print("logits_dim",logits_dim)
        super().__init__()
        self.latents = nn.Parameter(torch.randn(num_latents, latent_dim))
        self.cross_attend_blocks = nn.ModuleList([
            PreNorm(latent_dim, Attention(latent_dim, dim, heads = cross_heads, dim_head = cross_dim_head), context_dim = dim),
            PreNorm(latent_dim, FeedForward(latent_dim))
        ])
        get_latent_attn = lambda: PreNorm(latent_dim, Attention(latent_dim, heads = latent_heads, dim_head = latent_dim_head))
        get_latent_ff = lambda: PreNorm(latent_dim, FeedForward(latent_dim))
        get_latent_attn, get_latent_ff = map(cache_fn, (get_latent_attn, get_latent_ff))
        self.layers = nn.ModuleList([])
        cache_args = {'_cache': weight_tie_layers}
        for i in range(depth):
            self.layers.append(nn.ModuleList([
                get_latent_attn(**cache_args),
                get_latent_ff(**cache_args)
            ]))
        self.decoder_cross_attn = PreNorm(queries_dim, Attention(queries_dim, latent_dim, heads = cross_heads, dim_head = cross_dim_head), context_dim = latent_dim)
        self.decoder_ff = PreNorm(queries_dim, FeedForward(queries_dim)) if decoder_ff else None
        self.to_logits = nn.Linear(queries_dim, logits_dim) if exists(logits_dim) else nn.Identity()

    def forward(# self, data, mask = None, queries = None
        self, state, command, mask = None, queries = None):
        print("forward state, command",state.shape, command.shape)
        data = torch.cat((state, command),dim=-1)
        if data.dim()==1: data=data.unsqueeze(dim=0)
        elif data.dim() not in [1,2]: print("erm")
        data=data.unsqueeze(dim=1)
        print(data.shape)
        # data=data.view(1,1,data.size(dim=-1))
        print("forward state command data.shape",state.shape, command.shape,data.shape)# len 8 2 10

        b, *_, device = *data.shape, data.device
        x = repeat(self.latents, 'n d -> b n d', b = b)
        cross_attn, cross_ff = self.cross_attend_blocks
        # cross attention only happens once for Perceiver IO
        x = cross_attn(x, context = data, mask = mask) + x
        x = cross_ff(x) + x
        # layers
        for self_attn, self_ff in self.layers:
            x = self_attn(x) + x
            x = self_ff(x) + x
        if not exists(queries):
            return x
        # make sure queries contains batch dimension
        if queries.ndim == 2:
            queries = repeat(queries, 'n d -> b n d', b = b)
        # cross attend from decoder queries to latents
        latents = self.decoder_cross_attn(queries, context = x)
        # optional decoder feedforward
        if exists(self.decoder_ff):
            latents = latents + self.decoder_ff(latents)
        # final linear out
        return self.to_logits(latents)

    def action(self, state, desire, horizon):
        """Samples the action based on their probability"""
        command = torch.cat((desire*return_scale,horizon*horizon_scale), dim=-1)
        # print("actmion state",state," command ", command)
        action_prob = self.forward(state, command).squeeze()
        # print("action action_prob shape",action_prob)
        probs = torch.softmax(action_prob, dim=-1)
        # print("probs",probs)
        m = Categorical(probs)
        # print("m",m)
        action = m.sample()
        return action

    def greedy_action(self, state, desire, horizon):
        command = torch.cat((desire*return_scale,horizon*horizon_scale), dim=-1)
        action_prob = self.forward(state, command)
        probs = torch.softmax(action_prob, dim=-1)
        action = torch.argmax(probs).item()
        return action



# from perceiver_pytorch import PerceiverIO
model = PerceiverIO(
    dim = state_space,                    # dimension of sequence to be encoded
    queries_dim = 32,            # Q dimension of decoder queries
    logits_dim = action_space,            # dimension of final logits
    depth = 6,                   # depth of net
    num_latents = 256,           # N number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 1,            # D latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 64,         # number of dimensions per cross attention head
    latent_dim_head = 64,        # number of dimensions per latent self attention head
    weight_tie_layers = False    # whether to weight tie layers (optional, as indicated in the diagram)
    ).to(device)

seq = torch.randn(1, 1, 28*28, device=device) #(batch, latent_dim, dim)
queries = torch.randn(1, 32, device=device) #(decoder seq, queries_dim)

# logits = model(seq, queries = queries) # (batch, decoder seq, logits dim) range (-1,1)
# print(logits.shape) #torch.Size(batch, decoder seq, logits dim)
# logits=logits.squeeze()



class ReplayBuffer():
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        
    def add_sample(self, states, actions, rewards):
        episode = {"states": states, "actions":actions, "rewards": rewards, "summed_rewards":sum(rewards)}
        self.buffer.append(episode)
    
    def sort(self): #sort buffer
        self.buffer = sorted(self.buffer, key = lambda i: i["summed_rewards"],reverse=True)
        # keep the max buffer size
        self.buffer = self.buffer[:self.max_size]
    
    def get_random_samples(self, batch_size):
        self.sort()
        print("dgvjb",len(self.buffer), batch_size) #50 256
        idxs = np.random.randint(0, len(self.buffer), batch_size)
        batch = [self.buffer[idx] for idx in idxs]
        return batch
    
    def get_nbest(self, n):
        self.sort()
        return self.buffer[:n]
    
    def __len__(self):
        return len(self.buffer)

In [4]:

buffer = ReplayBuffer(replay_size)
bf = BF(state_space, action_space, 64, 1).to(device)
optimizer = optim.Adam(params=bf.parameters(), lr=1e-3)
# print(state_space, action_space) #8 4

# samples = []
# #initial command
init_desired_reward = 1 
init_time_horizon = 1 

# # for i in range(n_warm_up_episodes):
desired_return = torch.FloatTensor([init_desired_reward])
desired_time_horizon = torch.FloatTensor([init_time_horizon])
state = env.reset()
states = []
actions = []
rewards = []
# # while True:
# print(state, desired_return, desired_time_horizon)
# print("here",torch.from_numpy(state).float().to(device), desired_return.to(device), desired_time_horizon.to(device))
# # action = bf.action(torch.from_numpy(state).float().to(device), desired_return.to(device), desired_time_horizon.to(device))

# state=torch.from_numpy(state).float().to(device)
# desire=desired_return.to(device)
# horizon =desired_time_horizon.to(device)


state=torch.from_numpy(state).float().to(device)
desire=desired_return.to(device)
horizon =desired_time_horizon.to(device)


# action(self, state, desire, horizon):
"""Samples the action based on their probability"""
command = torch.cat((desire*return_scale,horizon*horizon_scale), dim=-1)
print("state command",state, command)


state = torch.randn(8, device=device)
command = torch.randn(2, device=device)
print("state command",state, command)


action_prob = bf(state, command).squeeze()
print("here action_prob",action_prob)
probs = torch.softmax(action_prob, dim=-1)
print("here probs",probs)
m = Categorical(probs)
action = m.sample()
print("here act",action,action_space)

next_state, reward, done, _ = env.step(action.cpu().numpy())
# states.append(torch.from_numpy(state).float())
states.append(state.float())
actions.append(action)
rewards.append(reward)
state = next_state
desired_return -= reward
desired_time_horizon -= 1
desired_time_horizon = torch.FloatTensor([np.maximum(desired_time_horizon, 1).item()])


dim 10
logits_dim None
state command tensor([-0.0021,  1.4040, -0.2096, -0.3092,  0.0024,  0.0475,  0.0000,  0.0000]) tensor([0.0200, 0.0200])
state command tensor([-1.8320, -0.5818,  0.6919,  0.5608,  1.6998,  0.0598, -0.4099, -0.0647]) tensor([ 0.4827, -0.1632])
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
here action_prob tensor([ 0.6949, -2.0306, -1.0026,  2.5602], grad_fn=<SqueezeBackward0>)
here probs tensor([0.1298, 0.0085, 0.0238, 0.8380], grad_fn=<SoftmaxBackward0>)
here act tensor(3) 4


In [5]:
# init replay buffer with n-warmup runs
buffer = ReplayBuffer(replay_size)
bf = BF(state_space, action_space, 64, 1).to(device)
optimizer = optim.Adam(params=bf.parameters(), lr=1e-3)
# print(state_space, action_space) #8 4


samples = []
#initial command
init_desired_reward = 1 
init_time_horizon = 1 

for i in range(n_warm_up_episodes):
    desired_return = torch.FloatTensor([init_desired_reward])
    desired_time_horizon = torch.FloatTensor([init_time_horizon])
    state = env.reset()
    states = []
    actions = []
    rewards = []
    while True:
        # print(state, desired_return, desired_time_horizon)
        action = bf.action(torch.from_numpy(state).float().to(device), desired_return.to(device), desired_time_horizon.to(device))
        print("act",action)
        next_state, reward, done, _ = env.step(action.cpu().numpy())
        states.append(torch.from_numpy(state).float())
        actions.append(action)
        rewards.append(reward)
        state = next_state
        desired_return -= reward
        desired_time_horizon -= 1
        desired_time_horizon = torch.FloatTensor([np.maximum(desired_time_horizon, 1).item()])
        if done:
            break 
    buffer.add_sample(states, actions, rewards)


Streaming output truncated to the last 5000 lines.
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
act tensor(0)
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
act tensor(2)
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
act tensor(2)
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
act tensor(2)
forward state, command torch.Size([8]) torch.Size([2])
torch.Size([1, 1, 10])
forward state command data.shape torch.Size([8]) torch.Size([2]) torch.Size([1, 1, 10])
act tensor(2)
forward state, command torch.Size([8]) torch.Size

In [6]:
## OBSERVE THE WEIGHTS before training
#for p in bf.parameters():
#    print(p)

In [7]:
#@title functions

# FUNCTIONS FOR Sampling exploration commands
def sampling_exploration( top_X_eps = last_few):
    """This function calculates the new desired reward and new desired horizon based on the replay buffer.
    New desired horizon is calculted by the mean length of the best last X episodes. 
    New desired reward is sampled from a uniform distribution given the mean and the std calculated from the last best X performances.
    where X is the hyperparameter last_few."""
    top_X = buffer.get_nbest(last_few)
    #The exploratory desired horizon dh0 is set to the mean of the lengths of the selected episodes
    new_desired_horizon = np.mean([len(i["states"]) for i in top_X])
    # save all top_X cumulative returns in a list 
    returns = [i["summed_rewards"] for i in top_X]
    # from these returns calc the mean and std
    mean_returns = np.mean(returns)
    std_returns = np.std(returns)
    # sample desired reward from a uniform distribution given the mean and the std
    new_desired_reward = np.random.uniform(mean_returns, mean_returns+std_returns)
    return torch.FloatTensor([new_desired_reward])  , torch.FloatTensor([new_desired_horizon])

# FUNCTIONS FOR TRAINING
def select_time_steps(saved_episode):
    """Given a saved episode from the replay buffer this function samples random time steps (t1 and t2) in that episode:
    T = max time horizon in that episode
    Returns t1, t2 and T """
    # Select times in the episode:
    T = len(saved_episode["states"]) # episode max horizon 
    t1 = np.random.randint(0,T-1)
    t2 = np.random.randint(t1+1,T)
    return t1, t2, T

def create_training_input(episode, t1, t2):
    """Based on the selected episode and the given time steps this function returns 4 values:
    1. state at t1
    2. the desired reward: sum over all rewards from t1 to t2
    3. the time horizont: t2 -t1
    4. the target action taken at t1
    buffer episodes are build like [cumulative episode reward, states, actions, rewards]"""
    state = episode["states"][t1] 
    desired_reward = sum(episode["rewards"][t1:t2])
    time_horizont = t2-t1
    action = episode["actions"][t1]
    return state, desired_reward, time_horizont, action

def create_training_examples(batch_size):
    """Creates a data set of training examples that can be used to create a data loader for training.
    ============================================================
    1. for the given batch_size episode idx are randomly selected
    2. based on these episodes t1 and t2 are samples for each selected episode 
    3. for the selected episode and sampled t1 and t2 trainings values are gathered
    ______________________________________________________________
    Output are two numpy arrays in the length of batch size:
    Input Array for the Behavior function - consisting of (state, desired_reward, time_horizon)
    Output Array with the taken actions """
    input_array = []
    output_array = []
    # select randomly episodes from the buffer
    episodes = buffer.get_random_samples(batch_size)
    for ep in episodes:
        t1, t2, T = select_time_steps(ep) #select time stamps
        t2 = T # For episodic tasks they set t2 to T
        state, desired_reward, time_horizont, action = create_training_input(ep, t1, t2)
        input_array.append(torch.cat([state, torch.FloatTensor([desired_reward]), torch.FloatTensor([time_horizont])]))
        output_array.append(action)
    return input_array, output_array

def train_behavior_function(batch_size):
    """Trains the BF with on a cross entropy loss were the inputs are the action probabilities based on the state and command.
    The targets are the actions appropriate to the states from the replay buffer."""
    X, y = create_training_examples(batch_size)
    X = torch.stack(X)
    state = X[:,0:state_space]
    d = X[:,state_space:state_space+1]
    h = X[:,state_space+1:state_space+2]
    command = torch.cat((d*return_scale,h*horizon_scale), dim=-1)
    y = torch.stack(y).long()
    print("in train",state.shape, command.shape)
    y_ = bf(state.to(device), command.to(device)).float()
    optimizer.zero_grad()
    pred_loss = F.cross_entropy(y_, y)   
    pred_loss.backward()
    optimizer.step()
    return pred_loss.detach().cpu().numpy()

def evaluate(desired_return = torch.FloatTensor([init_desired_reward]), desired_time_horizon = torch.FloatTensor([init_time_horizon])):
    """Runs one episode of the environment to evaluate the bf."""
    state = env.reset()
    rewards = 0
    while True:
        state = torch.FloatTensor(state)
        action = bf.action(state.to(device), desired_return.to(device), desired_time_horizon.to(device))
        state, reward, done, _ = env.step(action.cpu().numpy()) 
        rewards += reward
        desired_return = min(desired_return - reward, torch.FloatTensor([max_reward]))
        desired_time_horizon = max(desired_time_horizon - 1, torch.FloatTensor([1]))
        if done:
            break 
    return rewards

# Algorithm 2 - Generates an Episode unsing the Behavior Function:
def generate_episode(desired_return = torch.FloatTensor([init_desired_reward]), desired_time_horizon = torch.FloatTensor([init_time_horizon])):    
    """Generates more samples for the replay buffer."""
    state = env.reset()
    states = []
    actions = []
    rewards = []
    while True:
        state = torch.FloatTensor(state)
        action = bf.action(state.to(device), desired_return.to(device), desired_time_horizon.to(device))
        next_state, reward, done, _ = env.step(action.cpu().numpy())
        states.append(state)
        actions.append(action)
        rewards.append(reward)
        state = next_state
        desired_return -= reward
        desired_time_horizon -= 1
        desired_time_horizon = torch.FloatTensor([np.maximum(desired_time_horizon, 1).item()])
        if done:
            break 
    return [states, actions, rewards]


# Algorithm 1 - Upside - Down Reinforcement Learning 
def run_upside_down(max_episodes):
    all_rewards = []
    losses = []
    average_100_reward = []
    desired_rewards_history = []
    horizon_history = []
    for ep in range(1, max_episodes+1):
        # improve|optimize bf based on replay buffer
        loss_buffer = []
        for i in range(n_updates_per_iter):
            bf_loss = train_behavior_function(batch_size)
            loss_buffer.append(bf_loss)
        bf_loss = np.mean(loss_buffer)
        losses.append(bf_loss)
        # run x new episode and add to buffer
        for i in range(n_episodes_per_iter):
            # Sample exploratory commands based on buffer
            new_desired_reward, new_desired_horizon = sampling_exploration()
            generated_episode = generate_episode(new_desired_reward, new_desired_horizon)
            buffer.add_sample(generated_episode[0],generated_episode[1],generated_episode[2])
        new_desired_reward, new_desired_horizon = sampling_exploration()
        # monitoring desired reward and desired horizon
        desired_rewards_history.append(new_desired_reward.item())
        horizon_history.append(new_desired_horizon.item())
        ep_rewards = evaluate(new_desired_reward, new_desired_horizon)
        all_rewards.append(ep_rewards)
        average_100_reward.append(np.mean(all_rewards[-100:]))
        print("\rEpisode: {} | Rewards: {:.2f} | Mean_100_Rewards: {:.2f} | Loss: {:.2f}".format(ep, ep_rewards, np.mean(all_rewards[-100:]), bf_loss), end="", flush=True)
        if ep % 100 == 0:
            print("\rEpisode: {} | Rewards: {:.2f} | Mean_100_Rewards: {:.2f} | Loss: {:.2f}".format(ep, ep_rewards, np.mean(all_rewards[-100:]), bf_loss))
    return all_rewards, average_100_reward, desired_rewards_history, horizon_history, losses




In [8]:
%%time
rewards, average, d, h, loss = run_upside_down(max_episodes=200)
torch.save(bf.state_dict(), "behaviorfunction.pth")
plt.figure(figsize=(15,8))
plt.subplot(2,2,1)
plt.title("Rewards")
plt.plot(rewards, label="rewards")
plt.plot(average, label="average100")
plt.legend()
plt.subplot(2,2,2)
plt.title("Loss")
plt.plot(loss)
plt.subplot(2,2,3)
plt.title("desired Rewards")
plt.plot(d)
plt.subplot(2,2,4)
plt.title("desired Horizon")
plt.plot(h)
plt.show()

dgvjb 50 256
in train torch.Size([256, 8]) torch.Size([256, 2])
forward state, command torch.Size([256, 8]) torch.Size([256, 2])
torch.Size([256, 1, 10])
forward state command data.shape torch.Size([256, 8]) torch.Size([256, 2]) torch.Size([256, 1, 10])


RuntimeError: ignored

In [ ]:
# SAVE MODEL
name = "model.pth"
torch.save(bf.state_dict(), name)

## OBSERVE THE WEIGHTS after training
#for p in bf.parameters():
#    print(p)


In [ ]:

# evaluation
DESIRED_REWARD = torch.FloatTensor([200]).to(device)
DESIRED_HORIZON = torch.FloatTensor([200]).to(device)
desired = DESIRED_REWARD.item()

env = gym.make('CartPole-v0')
env.reset()
rewards = 0
while True:
    command = torch.cat((DESIRED_REWARD*return_scale,DESIRED_HORIZON*horizon_scale), dim=-1)
    probs_logits = bf(torch.from_numpy(state).float().to(device), command)
    probs = torch.softmax(probs_logits, dim=-1).detach().cpu()
    action = torch.argmax(probs).item()
    state, reward, done, info = env.step(action)
    rewards += reward
    DESIRED_REWARD -= reward
    DESIRED_HORIZON -= 1
    if done:
        break

print("Desired rewards: {} | after finishing one episode the agent received {} rewards".format(desired, rewards))
env.close()
